In [20]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

In [3]:
base_modelagem = pd.read_csv("bases_nba_stats/base_modelagem_nba_17_18.csv", index_col = 0)

In [42]:
base_modelagem.drop(["PLUS_MINUS_home",
                    "NET_RATING_away",                                   
                    "NET_RATING_home",                                    
                    "DEF_RATING_away",                                    
                    "OFF_RATING_home",                                  
                    "OFF_RATING_away",                                   
                    "DEF_RATING_home",                                   
                    "PTS_away",
                    "PTS_home"], axis=1, inplace=True)

In [43]:
correlations = abs(base_modelagem.corr()["fl_home_win"]).sort_values(ascending=False)

In [52]:
filtrada = base_modelagem[(base_modelagem.SEASON == 2018) & 
                           (base_modelagem.team_home_game_num > 5) & 
                           (base_modelagem.team_away_game_num > 5) &
                           (base_modelagem.fl_playoff == 0)]
len(filtrada)

1063

In [53]:
filtrada.fl_home_win.value_counts()/len(filtrada)

1    0.580433
0    0.419567
Name: fl_home_win, dtype: float64

In [57]:
list(filtrada.columns)

['index',
 'GAME_ID_home',
 'TEAM_ID_home',
 'TEAM_NAME_home',
 'team_home',
 'TEAM_CITY_home',
 'GAME_DATE_home',
 'GAME_PLACE_home',
 'TEAM_ID_away',
 'TEAM_NAME_away',
 'team_away',
 'TEAM_CITY_away',
 'DATE',
 'SEASON',
 'fl_playoff',
 'team_home_game_num',
 'team_away_game_num',
 'fl_home_win',
 'DISTANCE_KM_home',
 'DISTANCE_KM_away',
 'DAYS_FROM_LAST_GAME_home',
 'DAYS_FROM_LAST_GAME_away',
 'FGM_home_last_5_games',
 'FGA_home_last_5_games',
 'FG_PCT_home_last_5_games',
 'FG3M_home_last_5_games',
 'FG3A_home_last_5_games',
 'FG3_PCT_home_last_5_games',
 'FTM_home_last_5_games',
 'FTA_home_last_5_games',
 'FT_PCT_home_last_5_games',
 'OREB_home_last_5_games',
 'DREB_home_last_5_games',
 'REB_home_last_5_games',
 'AST_home_last_5_games',
 'STL_home_last_5_games',
 'BLK_home_last_5_games',
 'TO_home_last_5_games',
 'PF_home_last_5_games',
 'PTS_home_last_5_games',
 'PLUS_MINUS_home_last_5_games',
 'OFF_RATING_home_last_5_games',
 'DEF_RATING_home_last_5_games',
 'NET_RATING_home_la

In [58]:
pd.read_excel("descricao_colunas.xlsx")

,column_name,original_name,description,book
0,MIN,MIN,Minutes Played,traditional
1,FGM,FGM,Field Goals Made,traditional
2,FGA,FGA,Field Goals Attempted,traditional
3,FG_PCT,FG%,Field Goal Percentage,traditional
4,FG3M,3PM,Three Pointers Made,traditional
5,FG3A,3PA,Three Pointers Attempted,traditional
6,FG3_PCT,3P%,Three Point Percentage,traditional
7,FTM,FTM,Free Throws Made,traditional
8,FTA,FTA,Free Throws Attempted,traditional
9,FT_PCT,FT%,Free Throw Percentage,traditional


In [21]:
columns_subtract = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg']

for column in columns_subtract:
    filtrada["delta_" + column] = filtrada[column + "_home_last_5_games"] - filtrada[column + "_away_last_5_games"]
    filtrada["delta_opponent_home_" + column] = filtrada[column + "_opponent_home_last_5_games"] - filtrada[column + "_away_last_5_games"]
    filtrada["delta_opponent_away_" + column] = filtrada[column + "_opponent_away_last_5_games"] - filtrada[column + "_home_last_5_games"]

C:\Users\rafae\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\rafae\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rafae\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [25]:
print(list(filtrada.columns))

['team_home', 'PTS_home', 'team_away', 'PTS_away', 'date', 'season', 'team_home_game_num', 'team_away_game_num', 'fl_playoff', 'FG_home_last_5_games', 'FGA_home_last_5_games', 'FG%_home_last_5_games', '3P_home_last_5_games', '3PA_home_last_5_games', '3P%_home_last_5_games', 'FT_home_last_5_games', 'FTA_home_last_5_games', 'FT%_home_last_5_games', 'ORB_home_last_5_games', 'DRB_home_last_5_games', 'TRB_home_last_5_games', 'AST_home_last_5_games', 'STL_home_last_5_games', 'BLK_home_last_5_games', 'TOV_home_last_5_games', 'PF_home_last_5_games', 'PTS_home_last_5_games', 'TS%_home_last_5_games', 'eFG%_home_last_5_games', '3PAr_home_last_5_games', 'FTr_home_last_5_games', 'ORB%_home_last_5_games', 'DRB%_home_last_5_games', 'TRB%_home_last_5_games', 'AST%_home_last_5_games', 'STL%_home_last_5_games', 'BLK%_home_last_5_games', 'TOV%_home_last_5_games', 'USG%_home_last_5_games', 'ORtg_home_last_5_games', 'DRtg_home_last_5_games', 'FG_away_last_5_games', 'FGA_away_last_5_games', 'FG%_away_last_5

In [26]:
filtrada.head()

,team_home,PTS_home,team_away,PTS_away,date,season,team_home_game_num,team_away_game_num,fl_playoff,FG_home_last_5_games,...,delta_against_away_TOV%,delta_USG%,delta_against_home_USG%,delta_against_away_USG%,delta_ORtg,delta_against_home_ORtg,delta_against_away_ORtg,delta_DRtg,delta_against_home_DRtg,delta_against_away_DRtg
DEN @ MEM 2016-11-08,MEM,108.0,DEN,107.0,2016-11-08,2017,6,6,0,32.2,...,-3.64,0.0,0.0,0.0,-9.50,-1.50,10.28,-2.28,-10.28,1.50
DEN @ POR 2016-11-13,POR,112.0,DEN,105.0,2016-11-13,2017,6,7,0,37.8,...,-2.34,0.0,0.0,0.0,3.68,5.92,4.16,-1.92,-4.16,-5.92
ORL @ IND 2016-11-14,IND,88.0,ORL,69.0,2016-11-14,2017,6,6,0,42.0,...,-3.46,0.0,0.0,0.0,5.58,6.78,6.78,-5.58,-6.78,-6.78
BOS @ NOP 2016-11-14,NOP,106.0,BOS,105.0,2016-11-14,2017,6,6,0,39.6,...,2.28,0.0,0.0,0.0,-5.16,3.26,8.24,0.18,-8.24,-3.26
CHI @ POR 2016-11-15,POR,88.0,CHI,113.0,2016-11-15,2017,7,6,0,38.4,...,0.78,0.0,0.0,0.0,0.46,3.40,-4.90,7.84,4.90,-3.40


In [29]:
columns_to_drop = ['team_home', 'PTS_home', 'team_away', 'PTS_away', 'date',
                   'season', 'team_home_game_num', 'team_away_game_num', 'fl_playoff']
filtrada = filtrada.drop(columns_to_drop, axis=1)

In [41]:
base_modelo = filtrada.iloc[0:(len(filtrada) - 100)]

In [42]:
out_of_time = filtrada.iloc[(len(filtrada) - 100):len(filtrada)]

### Modelagem

In [115]:
from modelagem_ale import modelagem

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [50]:
X = base_modelo.drop("fl_home_win", axis=1)
y = base_modelo["fl_home_win"]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)
print(len(X_train), len(X_test))

575 384


In [111]:
data["FG%_home"] = [float(str(x).replace(",", ".")) for x in data["FG%_home"]]

0.123

In [116]:
modelagem = modelagem("class")

In [127]:
parametros = {
    "n_estimators": [18, 20, 22, 25],
    "max_depth": [13, 15],
    "min_samples_leaf": [7, 10, 15],
    "n_jobs": [2]
}

data, feat, cm_train,cm_test,model,grid = modelagem.model_experiment(parametros, 
                                                                       X_train, y_train.to_frame(), 
                                                                       X_test, y_test.to_frame(),
                                                                       'regular',
                                                                       'acc',
                                                                       'rf',
                                                                       cv = 5,
                                                                      )

Criando objeto do algoritmo escolhido ...
Iniciando treinamento ...
Avalidando modelo ...
Tempo de execução em h:m:s : 0:00:48


In [128]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
            oob_score=False, random_state=1234, verbose=0,
            warm_start=False)

In [129]:
data

,rf_Treino,rf_Teste
ROC,0.869804,0.587657
ACC,0.890435,0.611979
RECALL,0.976676,0.790698
PREC,0.858974,0.620438
F1,0.914052,0.695297
COHEN KAPPA,0.764673,0.182114
LOG LOSS,3.784325,13.401981


In [105]:
clf = GridSearchCV(RandomForestClassifier(), parametros, cv=5,
                       scoring='accuracy', n_jobs=2)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'n_estimators': [18, 20], 'max_depth': [15, 20, 25], 'min_samples_leaf': [7, 10, 15], 'n_jobs': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [106]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=15, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [130]:
y_true, y_pred = y_test, model.predict(X_test)
print(classification_report(y_true, y_pred, target_names=["away win", "home win"]))

             precision    recall  f1-score   support

   away win       0.59      0.38      0.47       169
   home win       0.62      0.79      0.70       215

avg / total       0.61      0.61      0.59       384



In [131]:
pd.DataFrame(confusion_matrix(y_true, y_pred))

,0,1
0,65,104
1,45,170


In [175]:
base_modelagem.date.min()

'2015-10-27'